# Grid Search
Exhaustive Hyperparameter Optimization with Cross-Validation
## Objective

This notebook introduces Grid Search as a systematic method for hyperparameter tuning. It focuses on:

- When grid search is appropriate

- How to design meaningful parameter grids

- Integrating grid search with pipelines
 
- Avoiding common tuning pitfalls

- Interpreting results beyond “best score”

It answers:

> _How do we tune models exhaustively without leaking data or wasting compute?_

## Why Grid Search Matters

Hyperparameters control:

- Model complexity

- Bias–variance tradeoff

- Regularization strength

- Learning dynamics

Poor defaults lead to:

- Overfitting

- Underfitting

- Unstable predictions

- Misleading evaluation metrics

Grid search provides deterministic, auditable optimization.

## When to Use Grid Search

- ✔ Small–to–medium search space
- ✔ Interpretable models
- ✔ Regulated environments
- ✔ Benchmarking and documentation

- ❌ Large parameter spaces
- ❌ Deep learning models
- ❌ Expensive training loops

## Imports and dataset

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import (
    GridSearchCV,
    train_test_split
)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [4]:
df = pd.read_csv("D:/GitHub/Data-Science-Techniques/datasets/Supervised-classification/synthetic_credit_default_classification.csv")

X = df.drop(columns=["default", "customer_id"])
y = df["default"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)


# Define a Leakage-Safe Pipeline

In [5]:
pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(
        max_iter=1000,
        solver="liblinear"
    ))
])


- ✔ Ensures preprocessing occurs inside CV folds
- ✔ Deployment-ready
- ✔ Reproducible

# Design the Parameter Grid

In [6]:
param_grid = {
    "model__penalty": ["l1", "l2"],
    "model__C": [0.01, 0.1, 1, 10],
    "model__class_weight": [None, "balanced"]
}


### Design Principles

- Grid size grows multiplicatively

- Each parameter should be theoretically justified

- Avoid blindly large grids

Grid size here:
- 2 × 4 × 2 = 16 combinations

# Configure GridSearchCV

In [7]:
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)


Key choices:

- **ROC AUC** → imbalance-robust

- **CV = 5** → bias–variance balance

- `return_train_score=True` → diagnose overfitting

## Fit Grid Search

In [8]:
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer',
                                        SimpleImputer(strategy='median')),
                                       ('scaler', StandardScaler()),
                                       ('model',
                                        LogisticRegression(max_iter=1000,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'model__C': [0.01, 0.1, 1, 10],
                         'model__class_weight': [None, 'balanced'],
                         'model__penalty': ['l1', 'l2']},
             return_train_score=True, scoring='roc_auc', verbose=1)

## Best Model and Parameters

In [9]:
grid_search.best_params_


{'model__C': 0.1, 'model__class_weight': None, 'model__penalty': 'l1'}

In [10]:
grid_search.best_score_


np.float64(0.9157104144658049)

This score is cross-validated, not test performance.

## Evaluate on Hold-Out Test Set

In [11]:
best_model = grid_search.best_estimator_

y_test_prob = best_model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_test_prob)


np.float64(0.9137766415257842)

- ✔ Honest generalization estimate
- ✔ No leakage

##  Analyze Full Grid Results

In [12]:
results = pd.DataFrame(grid_search.cv_results_)

results[
    [
        "params",
        "mean_test_score",
        "mean_train_score",
        "std_test_score"
    ]
].sort_values("mean_test_score", ascending=False)


,params,mean_test_score,mean_train_score,std_test_score
4,"{'model__C': 0.1, 'model__class_weight': None,...",0.915710,0.917515,0.010596
6,"{'model__C': 0.1, 'model__class_weight': 'bala...",0.915674,0.917524,0.010426
8,"{'model__C': 1, 'model__class_weight': None, '...",0.915613,0.917549,0.010625
7,"{'model__C': 0.1, 'model__class_weight': 'bala...",0.915610,0.917556,0.010589
10,"{'model__C': 1, 'model__class_weight': 'balanc...",0.915599,0.917561,0.010566
9,"{'model__C': 1, 'model__class_weight': None, '...",0.915597,0.917543,0.010633
13,"{'model__C': 10, 'model__class_weight': None, ...",0.915597,0.917543,0.010620
12,"{'model__C': 10, 'model__class_weight': None, ...",0.915592,0.917546,0.010628
5,"{'model__C': 0.1, 'model__class_weight': None,...",0.915583,0.917544,0.010615
14,"{'model__C': 10, 'model__class_weight': 'balan...",0.915575,0.917552,0.010563


## Bias–Variance Diagnostics

In [15]:
results["overfit_gap"] = (
    results["mean_train_score"] -
    results["mean_test_score"]
)

results.sort_values("overfit_gap", ascending=False).head()


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__C,param_model__class_weight,param_model__penalty,params,split0_test_score,split1_test_score,...,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,overfit_gap
2,0.022689,0.010854,0.007138,0.000372,0.01,balanced,l1,"{'model__C': 0.01, 'model__class_weight': 'bal...",0.918263,0.896426,...,0.010819,15,0.909228,0.914436,0.908340,0.910290,0.914683,0.911395,0.002657,0.002270
0,0.021881,0.004280,0.008636,0.003019,0.01,None,l1,"{'model__C': 0.01, 'model__class_weight': None...",0.917695,0.895255,...,0.010977,16,0.908053,0.913186,0.907009,0.909065,0.913442,0.910151,0.002664,0.002175
3,0.016653,0.000368,0.007122,0.000971,0.01,balanced,l2,"{'model__C': 0.01, 'model__class_weight': 'bal...",0.925539,0.904053,...,0.010466,13,0.914917,0.920720,0.914496,0.916894,0.920468,0.917499,0.002655,0.002007
11,0.013237,0.000595,0.005919,0.000582,1.00,balanced,l2,"{'model__C': 1, 'model__class_weight': 'balanc...",0.925530,0.904395,...,0.010573,12,0.914949,0.920743,0.914518,0.917018,0.920546,0.917555,0.002661,0.001989
1,0.017244,0.003253,0.015559,0.014872,0.01,None,l2,"{'model__C': 0.01, 'model__class_weight': None...",0.925593,0.904116,...,0.010347,14,0.914895,0.920626,0.914453,0.916853,0.920447,0.917455,0.002643,0.001984


- Large gap → potential overfitting
- Small gap + low score → underfitting

## Grid Search Cost Awareness

Approximate compute cost:

    # Fits = grid_size × CV folds
    16 × 5 = 80 model fits


## Common Mistakes (Avoided)

- ❌ Tuning on test set
- ❌ Scaling outside pipeline
- ❌ Overly large grids
- ❌ Ignoring overfitting signals
- ❌ Assuming “best params” = best business decision

## Summary Table

| Aspect                 | Grid Search |
| ---------------------- | ----------- |
| Search Type            | Exhaustive  |
| Reproducibility        | High        |
| Compute Cost           | High        |
| Interpretability       | Excellent   |
| Regulatory Suitability | Excellent   |


🟦 Key Takeaways

- Grid search is precise but expensive

- Pipelines are mandatory

- Grid size must be controlled

- Always inspect train vs validation

- Use grid search as a benchmark, not a default

🟦 Next Notebook

07_Model_Tuning_and_Optimization/

└── [02_randomized_search.ipynb](02_randomized_search.ipynb)